In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import string
import time

In [4]:
def get_titles():
    #Get all movies available at IMSDB.com and returns a Pandas Data Frame withe the titles.
    movies = {'title': []}
    alphabet = [str(0)] + list(string.ascii_uppercase) #List with all capital letter from A to Z and number 0

    for l in alphabet:
        url = 'http://www.imsdb.com/alphabetical/' +  l #URL for all movies starting with letter l
        r = requests.get(url)
        b = BeautifulSoup(r.text, 'html.parser')
        tag = b.findAll("p")
        for m in tag:
            movies["title"].append(m.find('a').contents[0]) #Movie title
    return pd.DataFrame(movies)

def get_script (title):
    #Receives a movie title as argument and get it script in IMSDB.com.
    url = 'http://www.imsdb.com/scripts/' + title.replace(' ', '-') + '.html' #URL for the movie script
    r = requests.get(url)
    b = BeautifulSoup(r.text, 'html.parser')
    tag = b.find("td", attrs={"class":'scrtext'})
    if tag:
        return tag.text
    else:
        return 'NULL'
    
def get_info (title):
    #Receives a movie title and info (ie. Director) as arguments and access the IMDB API to get the information.
    
    if ', the' in title:
        #Adjusts the title notation from IMSDB format to IMDB format.
        adjusted_title = 'the ' + title.replace(', the', '').replace(' ', '+')
    else:
        adjusted_title = title.replace(' ', '+')
    
    r = requests.get('http://www.omdbapi.com/?t=' + adjusted_title +   '&y=&plot=short&r=json')
    time.sleep(0.1)
    try:
        return r.json()
    except:
        return 'NULL'
    
def get_financials (ID):
    url = 'http://www.imdb.com/title/' +  ID 
    r = requests.get(url)
    b = BeautifulSoup(r.text, 'html.parser')
    titles = ['Budget' , 'Gross']
    financials = []
    for title in titles:
        financials.append(float(b.find(text=lambda x: x.startswith(title)).next_element.replace('$', '').replace(',','')))
    return financials[0], financials[1]


In [5]:
movies = get_titles()
movies['script'] = movies.title.apply(get_script)


AttributeError: 'module' object has no attribute 'movies'

In [10]:
movies.to_csv('movies.csv', encoding='utf-8')


In [13]:
movies = pd.read_csv('movies.csv')[['title', 'script']]

In [15]:
info = ['Actors',
        'imdbID',
        'Awards',
        'Country',
        'Director',
        'Genre',
        'Language',
        'Metascore',
        'Plot',
        'Rated',
        'Released',
        'Runtime',
        'Writer',
        'Year',
        'imdbRating',
        'imdbVotes']


movies['IMDB_info'] = movies.title.apply(lambda x: get_info(x))

In [17]:
movies.to_csv('movies.csv', encoding='utf-8')
movies = pd.read_csv('movies.csv')

In [18]:
movies.head()

,Unnamed: 0,title,script,IMDB_info
0,0,10 Things I Hate About You,\r\r\n\r\r\n\r\r\n\r\r\n<b><!--\r\r\n</b>if (w...,"{u'Plot': u'A pretty, popular teenager can't g..."
1,1,12,\r\r\n\r\r\n\r\r\n \r\r\n 12 - Script\r\r\n,{u'Plot': u'Twelve jurors must decide the fate...
2,2,12 and Holding,\r\r\n \r\r\n \r\r\n ...,{u'Plot': u'After his twin brother is accident...
3,3,12 Monkeys,\r\r\n\r\r\n\r\r\n\r\r\n<b><!--\r\r\n</b>if (w...,"{u'totalSeasons': u'2', u'Plot': u'Follows the..."
4,4,12 Years a Slave,\r\r\n\r\r\r\n\r\r\r\n\r\r\r\n ...,"{u'Plot': u'In the antebellum United States, S..."


In [25]:
import numpy as np
movies[movies.script.isnull()]

,Unnamed: 0,title,script,IMDB_info
24,24,A.I.,NaN,{u'Plot': u'A highly advanced robotic boy long...
135,135,Batman and Robin,NaN,{u'Plot': u'The caped crusaders versus The Wiz...
136,136,Batman Begins,NaN,"{u'Plot': u'After training with his mentor, Ba..."
137,137,Batman Forever,NaN,{u'Plot': u'Batman must battle Two-Face and Th...
138,138,Batman Returns,NaN,{u'Plot': u'When a corrupt businessman and the...
222,222,Casablanca,NaN,"{u'Plot': u'In Casablanca, Morocco in December..."
268,268,Contact,NaN,"{u'Plot': u'Dr. Ellie Arroway, after years of ..."
297,297,"Dark Knight, The",NaN,{u'Plot': u'A look at the history of the Batma...
332,332,Donnie Darko,NaN,{u'Plot': u'A troubled teenager is plagued by ...
371,371,Executive Decision,NaN,{u'Plot': u'When terrorists seize control of a...
